# Hybrid Search with FastEmbed & Qdrant

Author: [Nirant Kasliwal](https://twitter.com/nirantk)

## What will we do?
This notebook demonstrates the usage of Hybrid Search with FastEmbed & Qdrant. 

1. Setup: Download and install the required dependencies
2. Preview data: Load and preview the data
3. Create Sparse Embeddings: Create SPLADE++ embeddings for the data
4. Create Dense Embeddings: Create BGE-Large-en-v1.5 embeddings for the data
5. Indexing: Index the embeddings using Qdrant
6. Search: Perform Hybrid Search using FastEmbed & Qdrant
7. Ranking: Rank the search results with Reciprocal Rank Fusion (RRF)

## Setup

In order to get started, you need a few dependencies, and we'll install them next:

In [ ]:
!pip install -qU qdrant-client fastembed datasets transformers

In [29]:
import json
from typing import List, Tuple

import numpy as np
import pandas as pd
from datasets import load_dataset
from qdrant_client import QdrantClient
from qdrant_client.models import (
    Distance,
    NamedSparseVector,
    NamedVector,
    SparseVector,
    PointStruct,
    SearchRequest,
    SparseIndexParams,
    SparseVectorParams,
    VectorParams,
    ScoredPoint,
)
from transformers import AutoTokenizer

import fastembed
from fastembed import SparseEmbedding, SparseTextEmbedding, TextEmbedding

In [30]:
fastembed.__version__  # 0.2.5

'0.2.5'

In [31]:
dataset = load_dataset("tasksource/esci", split="train")
# We'll select the first 1000 examples for this demo
dataset = dataset.select(range(1000))
dataset = dataset.filter(lambda x: x["product_locale"] == "us")
dataset

Dataset({
    features: ['example_id', 'query', 'query_id', 'product_id', 'product_locale', 'esci_label', 'small_version', 'large_version', 'product_title', 'product_description', 'product_bullet_point', 'product_brand', 'product_color', 'product_text'],
    num_rows: 919
})

## Preview Data

In [32]:
source_df = dataset.to_pandas()
df = source_df.drop_duplicates(subset=["product_text", "product_title", "product_bullet_point", "product_brand"])
df = df.dropna(subset=["product_text", "product_title", "product_bullet_point", "product_brand"])
df.head()

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,product_title,product_description,product_bullet_point,product_brand,product_color,product_text
0,0,revent 80 cfm,0,B000MOO21W,us,Irrelevant,0,1,Panasonic FV-20VQ3 WhisperCeiling 190 CFM Ceil...,None,WhisperCeiling fans feature a totally enclosed...,Panasonic,White,Panasonic FV-20VQ3 WhisperCeiling 190 CFM Ceil...
2,1,revent 80 cfm,0,B07X3Y6B1V,us,Exact,0,1,Homewerks 7141-80 Bathroom Fan Integrated LED ...,None,OUTSTANDING PERFORMANCE: This Homewerk's bath ...,Homewerks,80 CFM,Homewerks 7141-80 Bathroom Fan Integrated LED ...
3,2,revent 80 cfm,0,B07WDM7MQQ,us,Exact,0,1,Homewerks 7140-80 Bathroom Fan Ceiling Mount E...,None,OUTSTANDING PERFORMANCE: This Homewerk's bath ...,Homewerks,White,Homewerks 7140-80 Bathroom Fan Ceiling Mount E...
4,3,revent 80 cfm,0,B07RH6Z8KW,us,Exact,0,1,Delta Electronics RAD80L BreezRadiance 80 CFM ...,This pre-owned or refurbished product has been...,Quiet operation at 1.5 sones\nBuilt-in thermos...,DELTA ELECTRONICS (AMERICAS) LTD.,White,Delta Electronics RAD80L BreezRadiance 80 CFM ...
5,4,revent 80 cfm,0,B07QJ7WYFQ,us,Exact,0,1,Panasonic FV-08VRE2 Ventilation Fan with Reces...,None,The design solution for Fan/light combinations...,Panasonic,White,Panasonic FV-08VRE2 Ventilation Fan with Reces...


In [33]:
print(f"Catalog Item Count: {len(df)}\nQueries: {len(source_df)}")

Catalog Item Count: 176
Queries: 919


In [34]:
df["combined_text"] = df["product_title"] + "\n" + df["product_text"] + "\n" + df["product_bullet_point"]

In [35]:
len(df)

176

## Create Sparse Embeddings

In [ ]:
sparse_model_name = "prithvida/Splade_PP_en_v1"
dense_model_name = "BAAI/bge-large-en-v1.5"
# This triggers the model download
sparse_model = SparseTextEmbedding(model_name=sparse_model_name, batch_size=32)
dense_model = TextEmbedding(model_name=dense_model_name, batch_size=32)

In [37]:
def make_sparse_embedding(texts: List[str]):
    return list(sparse_model.embed(texts))


sparse_embedding: List[SparseEmbedding] = make_sparse_embedding(["Fastembed is a great library for text embeddings!"])
sparse_embedding

[SparseEmbedding(values=array([0.17295611, 0.80484658, 0.41356239, 0.38512513, 0.90825951,
        0.61373132, 0.18313883, 0.18546289, 0.04257777, 1.20476401,
        1.48403311, 0.17008089, 0.06487759, 0.16780744, 0.23214206,
        2.5722568 , 1.87174428, 0.2541669 , 0.20749982, 0.16315481,
        0.70712435, 0.26381177, 0.49152234, 0.67282563, 0.19267203,
        0.29127747, 0.09682107, 1.21251154, 0.19741221, 0.44512141,
        0.44369081, 0.21676107, 0.36704862, 0.06706504, 1.97674787,
        0.00856015, 0.51626593, 0.21145488, 0.09790635, 0.26357391,
        1.6925323 , 2.10766435, 0.05584541, 0.05150893, 0.24062614,
        0.90479541, 0.1198509 , 0.10030396]), indices=array([ 1037,  2003,  2005,  2190,  2204,  2307,  2338,  2497,  2565,
         2793,  3075,  3177,  3274,  3286,  3430,  3435,  3793,  3819,
         3835,  3989,  4007,  4118,  4248,  4289,  4294,  4322,  4434,
         4667,  4773,  5080,  5371,  5527,  6028,  6581,  6633,  6919,
         6981,  6994,  7809,

The previous output is a SparseEmbedding object for the first document in our list.

It contains two arrays: values and indices. 
- The 'values' array represents the weights of the features (tokens) in the document.
- The 'indices' array represents the indices of these features in the model's vocabulary.

Each pair of corresponding values and indices represents a token and its weight in the document.

This is still a little abstract, so let's use the tokenizer vocab to make sense of these indices.

In [38]:
SparseTextEmbedding.list_supported_models()

[{'model': 'prithvida/Splade_PP_en_v1',
  'vocab_size': 30522,
  'description': 'Misspelled version of the model. Retained for backward compatibility. Independent Implementation of SPLADE++ Model for English',
  'size_in_GB': 0.532,
  'sources': {'hf': 'Qdrant/SPLADE_PP_en_v1'}},
 {'model': 'prithivida/Splade_PP_en_v1',
  'vocab_size': 30522,
  'description': 'Independent Implementation of SPLADE++ Model for English',
  'size_in_GB': 0.532,
  'sources': {'hf': 'Qdrant/SPLADE_PP_en_v1'}}]

In [39]:
def get_tokens_and_weights(sparse_embedding, model_name):
    # Find the tokenizer for the model
    tokenizer_source = None
    for model_info in SparseTextEmbedding.list_supported_models():
        if model_info["model"].lower() == model_name.lower():
            tokenizer_source = model_info["sources"]["hf"]
            break
        else:
            raise ValueError(f"Model {model_name} not found in the supported models.")

    tokenizer = AutoTokenizer.from_pretrained(tokenizer_source)
    token_weight_dict = {}
    for i in range(len(sparse_embedding.indices)):
        token = tokenizer.decode([sparse_embedding.indices[i]])
        weight = sparse_embedding.values[i]
        token_weight_dict[token] = weight

    # Sort the dictionary by weights
    token_weight_dict = dict(sorted(token_weight_dict.items(), key=lambda item: item[1], reverse=True))
    return token_weight_dict


# Test the function with the first SparseEmbedding
print(json.dumps(get_tokens_and_weights(sparse_embedding[0], sparse_model_name), indent=4))

{
    "fast": 2.5722568035125732,
    "##bed": 2.1076643466949463,
    "##em": 1.9767478704452515,
    "text": 1.8717442750930786,
    "em": 1.6925323009490967,
    "library": 1.4840331077575684,
    "##ding": 1.2125115394592285,
    "bed": 1.2047640085220337,
    "good": 0.9082595109939575,
    "librarian": 0.9047954082489014,
    "is": 0.8048465847969055,
    "software": 0.7071243524551392,
    "format": 0.6728256344795227,
    "great": 0.613731324672699,
    "texts": 0.5162659287452698,
    "quick": 0.49152234196662903,
    "device": 0.4451214075088501,
    "file": 0.44369080662727356,
    "for": 0.4135623872280121,
    "best": 0.38512513041496277,
    "technique": 0.36704862117767334,
    "facility": 0.2912774682044983,
    "method": 0.26381176710128784,
    "ideal": 0.26357391476631165,
    "perfect": 0.2541669011116028,
    "##bing": 0.24062614142894745,
    "material": 0.23214206099510193,
    "storage": 0.21676106750965118,
    "tool": 0.21145488321781158,
    "nice": 0.2074998

## Create Dense Embeddings

In [40]:
def make_dense_embedding(texts: List[str]):
    return list(dense_model.embed(texts))


dense_embedding = make_dense_embedding(["Fastembed is a great library for text embeddings!"])

In [41]:
dense_embedding[0].shape

(1024,)

In [42]:
product_texts = df["combined_text"].tolist()

In [ ]:
%%time
df["sparse_embedding"] = make_sparse_embedding(product_texts)

Notice that FastEmbed uses data parallelism to speed up the embedding generation process. This improves throughput and reduces the time it takes to generate embeddings for large datasets. For our small dataset here, it reduces the time from 5 min 15 seconds to 1 min 44 seconds, or about 3x faster.

In [44]:
df["sparse_embedding"]

0      SparseEmbedding(values=array([0.06509431, 0.57...
2      SparseEmbedding(values=array([0.10595927, 0.20...
3      SparseEmbedding(values=array([0.1140037 , 0.02...
4      SparseEmbedding(values=array([6.13510251e-01, ...
5      SparseEmbedding(values=array([0.90058267, 0.12...
                             ...                        
780    SparseEmbedding(values=array([5.56782305e-01, ...
809    SparseEmbedding(values=array([0.38585788, 0.44...
828    SparseEmbedding(values=array([3.27695787e-01, ...
867    SparseEmbedding(values=array([0.36255798, 0.74...
870    SparseEmbedding(values=array([3.74321818e-01, ...
Name: sparse_embedding, Length: 176, dtype: object

In [ ]:
%%time
df["dense_embedding"] = make_dense_embedding(product_texts)

## Indexing

In [46]:
client = QdrantClient(":memory:")

Qdrant is a vector similarity search engine that allows you to index and search high-dimensional vectors. It supports both sparse and dense embeddings, and it's a great tool for building search engines. Here, we use the memory mode which is Numpy under the hood for demonstration purposes. In production, you can use the Docker or Cloud for full DB support.

In [47]:
collection_name = "esci"
client.create_collection(
    collection_name,
    vectors_config={
        "text-dense": VectorParams(
            size=1024,  # OpenAI Embeddings
            distance=Distance.COSINE,
        )
    },
    sparse_vectors_config={
        "text-sparse": SparseVectorParams(
            index=SparseIndexParams(
                on_disk=False,
            )
        )
    },
)

True

In [48]:
def make_points(df: pd.DataFrame) -> List[PointStruct]:
    sparse_vectors = df["sparse_embedding"].tolist()
    product_texts = df["combined_text"].tolist()
    dense_vectors = df["dense_embedding"].tolist()
    rows = df.to_dict(orient="records")
    points = []
    for idx, (text, sparse_vector, dense_vector) in enumerate(zip(product_texts, sparse_vectors, dense_vectors)):
        sparse_vector = SparseVector(indices=sparse_vector.indices.tolist(), values=sparse_vector.values.tolist())
        point = PointStruct(
            id=idx,
            payload={"text": text, "product_id": rows[idx]["product_id"]},  # Add any additional payload if necessary
            vector={
                "text-sparse": sparse_vector,
                "text-dense": dense_vector,
            },
        )
        points.append(point)
    return points


points: List[PointStruct] = make_points(df)

In [49]:
client.upsert(collection_name, points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

## Search

In [50]:
def search(query_text: str):
    # # Compute sparse and dense vectors
    query_sparse_vectors: List[SparseEmbedding] = make_sparse_embedding([query_text])
    query_dense_vector: List[np.ndarray] = make_dense_embedding([query_text])

    search_results = client.search_batch(
        collection_name=collection_name,
        requests=[
            SearchRequest(
                vector=NamedVector(
                    name="text-dense",
                    vector=query_dense_vector[0],
                ),
                limit=10,
                with_payload=True,
            ),
            SearchRequest(
                vector=NamedSparseVector(
                    name="text-sparse",
                    vector=SparseVector(
                        indices=query_sparse_vectors[0].indices.tolist(),
                        values=query_sparse_vectors[0].values.tolist(),
                    ),
                ),
                limit=10,
                with_payload=True,
            ),
        ],
    )

    return search_results


query_text = " revent 80 cfm"
search_results = search(query_text)

## Ranking

We'll combine the results from the two models using Reciprocal Rank Fusion (RRF). You can read more about RRF [here](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf).

We select RRF for this task because:
1. It is a simple and effective method for combining search results.
2. It is robust to the differences in the ranking scores of the two or more ranking lists.
3. It is easy to implement and requires minimal tuning (only one parameter: alpha, which we don't tune here).

In [51]:
def rrf(rank_lists, alpha=60, default_rank=1000):
    """
    Optimized Reciprocal Rank Fusion (RRF) using NumPy for large rank lists.

    :param rank_lists: A list of rank lists. Each rank list should be a list of (item, rank) tuples.
    :param alpha: The parameter alpha used in the RRF formula. Default is 60.
    :param default_rank: The default rank assigned to items not present in a rank list. Default is 1000.
    :return: Sorted list of items based on their RRF scores.
    """
    # Consolidate all unique items from all rank lists
    all_items = set(item for rank_list in rank_lists for item, _ in rank_list)

    # Create a mapping of items to indices
    item_to_index = {item: idx for idx, item in enumerate(all_items)}

    # Initialize a matrix to hold the ranks, filled with the default rank
    rank_matrix = np.full((len(all_items), len(rank_lists)), default_rank)

    # Fill in the actual ranks from the rank lists
    for list_idx, rank_list in enumerate(rank_lists):
        for item, rank in rank_list:
            rank_matrix[item_to_index[item], list_idx] = rank

    # Calculate RRF scores using NumPy operations
    rrf_scores = np.sum(1.0 / (alpha + rank_matrix), axis=1)

    # Sort items based on RRF scores
    sorted_indices = np.argsort(-rrf_scores)  # Negative for descending order

    # Retrieve sorted items
    sorted_items = [(list(item_to_index.keys())[idx], rrf_scores[idx]) for idx in sorted_indices]

    return sorted_items


# Example usage
rank_list1 = [("A", 1), ("B", 2), ("C", 3)]
rank_list2 = [("B", 1), ("C", 2), ("D", 3)]
rank_list3 = [("A", 2), ("D", 1), ("E", 3)]

# Combine the rank lists
sorted_items = rrf([rank_list1, rank_list2, rank_list3])
sorted_items

[('B', 0.033465871107430434),
 ('A', 0.033465871107430434),
 ('D', 0.03320985472238179),
 ('C', 0.03294544435749548),
 ('E', 0.01775980832584606)]

Based on this, let's convert our sparse and dense results into rank lists. And then, we'll use the Reciprocal Rank Fusion (RRF) algorithm to combine them.

In [52]:
def rank_list(search_result: List[ScoredPoint]):
    return [(point.id, rank + 1) for rank, point in enumerate(search_result)]


dense_rank_list, sparse_rank_list = rank_list(search_results[0]), rank_list(search_results[1])

In [53]:
rrf_rank_list = rrf([dense_rank_list, sparse_rank_list])

In [54]:
rrf_rank_list

[(3, 0.032018442622950824),
 (8, 0.03149801587301587),
 (1, 0.03131881575727918),
 (13, 0.030834914611005692),
 (15, 0.030536130536130537),
 (9, 0.030309988518943745),
 (12, 0.030158730158730156),
 (14, 0.029437229437229435),
 (11, 0.028985507246376812),
 (2, 0.01707242848447961),
 (4, 0.01564927857935627)]

In [55]:
def find_point_by_id(client: QdrantClient, collection_name: str, rrf_rank_list: List[Tuple[int, float]]):
    return client.retrieve(collection_name=collection_name, ids=[item[0] for item in rrf_rank_list])


find_point_by_id(client, collection_name, rrf_rank_list)

[Record(id=3, payload={'text': 'Delta Electronics RAD80L BreezRadiance 80 CFM Heater/Fan/Light Combo White (Renewed)\nDelta Electronics RAD80L BreezRadiance 80 CFM Heater/Fan/Light Combo White (Renewed)\nDELTA ELECTRONICS (AMERICAS) LTD.\nWhite\nThis pre-owned or refurbished product has been professionally inspected and tested to work and look like new. How a product becomes part of Amazon Renewed, your destination for pre-owned, refurbished products: A customer buys a new product and returns it or trades it in for a newer or different model. That product is inspected and tested to work and look like new by Amazon-qualified suppliers. Then, the product is sold as an Amazon Renewed product on Amazon. If not satisfied with the purchase, renewed products are eligible for replacement or refund under the Amazon Renewed Guarantee.\nQuiet operation at 1.5 sones\nBuilt-in thermostat regulates temperature. Energy efficiency at 7.6 CFM/Watt\nPrecision engineered with DC brushless motor for exten

Next, let's check the ESCI (Exact, Substitute, Compliment, and Irrelvant) label for the results against the source data.

In [56]:
ids = [item[0] for item in rrf_rank_list]
df[df["query"] == query_text]

for idx in ids:
    print(df.iloc[idx]["esci_label"])

Exact
Exact
Exact
Exact
Exact
Exact
Exact
Exact
Exact
Exact
Exact


This was amazing! We pulled only Exact results with k=10. This is a great result for a small dataset like this with out of the box vectors which are not even fine-tuned for e-Commerce.

In [57]:
len(rrf_rank_list)

11

## Conclusion

In this notebook, we demonstrated the usage of Hybrid Search with FastEmbed & Qdrant. We used FastEmbed to create Sparse and Dense embeddings for the data and indexed them using Qdrant. We then performed Hybrid Search using FastEmbed & Qdrant and ranked the search results using Reciprocal Rank Fusion (RRF).